# PROJECT LAYOUT

#### Aim: To filter out colleges from all over the US according to Maria's preferences, which are that the colleges must:
#### 1. be in an urban(metro) area
#### 2. be in a city that ranks 75th percentile or higher on Kauffman's start-up rankings.
#### 3. be below 50th percentile in overall crime.
#### 4. offer a 2-year or 4-year degree in Information Technology/Science.

# A. Startup Data

#### Because Maria wants colleges to be in Metro cities, a filtering on that basis would leave a smaller data set
#### in the other two files, making the analysis convenient

In [62]:
## Import Libraries

import pandas as pd
import numpy as np

In [63]:
startup = pd.read_csv("Startup Activity.csv")
startup.head()

,Location,Index Year,Breakdown,Characteristic,Rank,z_rate,z_oppshare,z_startup_den,norm_rate,norm_opp,...,rank_diff,index_last_y,index_diff,Location_name,Category,OppShare_New_Entrp_web,Rate_New_Entrp_web,location_category,index_diff_text,dataset
0,0,1996.0,Education,Less than High School,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,National,NaN,393.46829,the,.,national_demog
1,0,1998.0,Age,Ages 35-44,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,National,7.9575,305.42319,the,.,national_demog
2,0,2011.0,Race,White,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,National,7.6008,293.58240,the,.,national_demog
3,0,2011.0,Race,Other,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,National,NaN,NaN,the,.,national_demog
4,0,2011.0,Race,Latino,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,National,6.7777,515.29340,the,.,national_demog


### Cleaning the Data

In [64]:
# Since the data is for year 2015 filtering for 2015
# And Dropping irrelevant columns
startup = startup[["Index Year", "Rank", "Location_name"]][startup["Index Year"] == 2015]

#Dropping "Index Year" column, since it contains the single value '2015'.
startup = startup.drop("Index Year", axis=1)

# Data file indicates presence of states in the city column as well.
#Fetching values to remove that
startup["Location_name"].unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
       'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming',
       'Atlanta-Sandy Springs-Marietta', 'Austin-Round Rock-San Marcos',
       'Baltimore-Towson', 'Boston-Cambridge-Quincy',
       'Charlotte-Gastonia-Rock Hill', 'Chicago-Joliet-Naperville',
       'Cincinnati-Middletown', 'Cleveland-Elyria-Mentor', 'Columbus',
       'Dallas-For

In [65]:
# Removing country and state names

states = ["United States", "Alabama", "Alaska", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

for name in states:
    startup = startup[startup["Location_name"] != name]

### Selecting top 25 percentile cities

In [66]:
## Sorting cities by their Startup Activity ranks

startup = startup.sort_values(by="Rank")
startup.shape

(40, 2)

In [67]:
# Selecting top 25 percentile values.
# Since the number of values is 40, top 25 percentile are the top 10 values
startup = startup[:10]
startup

,Rank,Location_name
2413,1.0,Austin-Round Rock-San Marcos
3042,2.0,Miami-Fort Lauderdale-Pompano Beach
3683,3.0,San Jose-Sunnyvale-Santa Clara
2991,4.0,Los Angeles-Long Beach-Santa Ana
2730,5.0,Denver-Aurora-Broomfield
2965,6.0,Las Vegas-Paradise
2821,7.0,Houston-Sugar Land-Baytown
3632,8.0,San Francisco-Oakland-Fremont
3574,9.0,San Antonio-New Braunfels
3194,10.0,New York-Northern New Jersey-Long Island


### Fetching Metro area names

In [68]:
## Since Maria prefers Metro areas, which can be seen as the first city names before the first '-' delimeter,
## Extracting the Metro names only

A = startup["Location_name"].str.split("-", expand=True)
A

,0,1,2
2413,Austin,Round Rock,San Marcos
3042,Miami,Fort Lauderdale,Pompano Beach
3683,San Jose,Sunnyvale,Santa Clara
2991,Los Angeles,Long Beach,Santa Ana
2730,Denver,Aurora,Broomfield
2965,Las Vegas,Paradise,None
2821,Houston,Sugar Land,Baytown
3632,San Francisco,Oakland,Fremont
3574,San Antonio,New Braunfels,None
3194,New York,Northern New Jersey,Long Island


In [69]:
# Concatenating the dataframe containing split names with startup dataframe
startup = pd.concat([A, startup], axis=1)

startup.columns = ["city", "C", "D", "Rank", "Location_name"]

# Keeping only the Metro city name
startup["city"] = startup["city"].astype(str)

startup = startup[["city", "Rank"]]

startup.columns = ["city", "Startup_Rank"]
startup

,city,Startup_Rank
2413,Austin,1.0
3042,Miami,2.0
3683,San Jose,3.0
2991,Los Angeles,4.0
2730,Denver,5.0
2965,Las Vegas,6.0
2821,Houston,7.0
3632,San Francisco,8.0
3574,San Antonio,9.0
3194,New York,10.0


# B. CRIME DATA

In [105]:
crime = pd.read_csv("crime_2015.csv")

### Data Cleaning

In [106]:
## Dropping irrelevant columns
# violent crime and property crime are sums of individual crime categories

crime = crime.drop(["msa", "violentcrime", "propertycrime", "state"], axis=1)

# Converting all the columns to numeric forms

cols = crime.columns.drop("city")

for col in cols:
    crime[col] = pd.to_numeric(crime[col], errors="coerce")
    
crime.head()

,murder,rape,robbery,aggravatedassault,burglary,theft,motorvehicletheft,city
0,5.3,56.0,78.4,272.8,852.0,2493.6,263.4,Abilene
1,5.1,38.2,75.2,119.8,575.3,1853.0,124.1,Akron
2,7.8,30.4,157.9,471.8,1099.6,2652.8,141.7,Albany
3,2.5,28.2,20.7,63.0,484.6,2476.1,247.7,Albany
4,6.1,63.8,206.7,516.0,883.4,3047.6,676.9,Albuquerque


In [107]:
crime = crime.fillna(crime.mean())

### Filtering crime dataframe for Metro cities

In [108]:
Metro_cities = ["Austin", "Miami", "San Jose", "Los Angeles", "Denver", "Las Vegas", "Houston", "San Francisco", "San Antonio", "New York"]

crime = crime[crime["city"].isin(Metro_cities)]

### Creating a CRIME ranking metric

In [109]:
# For this, I assign following weightage to individual crime categories:

#murder - 25% - serious
#rape - 20% - serious(Maria is a woman)
#robbery - 10%
#assault - 15% - moderately serious
#burglary - 10%
#theft - 5%
#mvtheft - 15% - moderately serious

crime["city_crime_score"] = 0.25*crime["murder"] + 0.2*crime["rape"] + 0.1*crime["robbery"] + 0.15*crime["aggravatedassault"] + 0.1*crime["burglary"] + 0.05*crime["theft"] + 0.15*crime["motorvehicletheft"]

## Filtering crime only for crime and crime metric

crime = crime[["city", "city_crime_score"]]

crime

,city,city_crime_score
17,Austin,213.000000
91,Denver,260.676859
149,Houston,285.879640
184,Las Vegas,341.755000
196,Los Angeles,235.255000
198,Los Angeles,251.465000
213,Miami,289.945000
215,Miami,326.525000
294,San Antonio,326.060000
296,San Francisco,332.130000


### Removing Duplicates and finding mean for repeated values

In [110]:
## Averaging the values for LA, SF and Miami values (Since they fall into two jurisdictions, they come twice)
## Crime score to be averaged, duplicate entry to be dropped

crime = crime.append({"city": "LA", "city_crime_score": 243.33}, ignore_index=True)
crime = crime.append({"city": "MIA", "city_crime_score": 308.24}, ignore_index=True)
crime = crime.append({"city": "SF", "city_crime_score": 346.63}, ignore_index=True)

crime = crime[crime["city"] != "Los Angeles"]
crime = crime[crime["city"] != "Miami"]
crime = crime[crime["city"] != "San Francisco"]

crime["city"] = crime["city"].replace({"LA": "Los Angeles", "MIA": "Miami", "SF": "San Francisco"})

crime

,city,city_crime_score
0,Austin,213.000000
1,Denver,260.676859
2,Houston,285.879640
3,Las Vegas,341.755000
8,San Antonio,326.060000
11,San Jose,220.360000
12,Los Angeles,243.330000
13,Miami,308.240000
14,San Francisco,346.630000


### Assigning Ranks and finalizing Crime Data

In [111]:
## Assigning ranks by sorting by values and then using indexes

crime = crime.sort_values(by="city_crime_score", ascending=True)

crime = crime.reset_index()

crime["Safety_Crime_Rank"] = crime["city_crime_score"].index + 1

crime = crime.drop(["index", "city_crime_score"], axis=1)

crime

,city,Safety_Crime_Rank
0,Austin,1
1,San Jose,2
2,Los Angeles,3
3,Denver,4
4,Houston,5
5,Miami,6
6,San Antonio,7
7,Las Vegas,8
8,San Francisco,9


# C. PREPARING COLLEGE DATA

In [259]:
college = pd.read_csv("collegescorecard.csv")

college.head()

C:\Users\hp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,unitid,opeid,opeid6,instnm,city,stabbr,zip,accredagency,insturl,npcurl,...,nopell_rpy_3yr_rt_supp,female_rpy_3yr_rt_supp,male_rpy_3yr_rt_supp,firstgen_rpy_3yr_rt_supp,notfirstgen_rpy_3yr_rt_supp,c150_l4_pooled_supp,c150_4_pooled_supp,c200_l4_pooled_supp,c200_4_pooled_supp,location
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,galileo.aamu.edu/netpricecalculator/npcalc.htm,...,0.45251396648,0.484818805093,0.406,0.423580786026,0.442452830189,NaN,0.30871831265508,NaN,0.35168556430446,POINT(34.7834 -86.5685)
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,www.collegeportraits.org/AL/UAB/estimator/agree,...,0.795454545455,0.760521885522,0.755344418052,0.751552795031,0.765074770863,NaN,0.50854980789381,NaN,0.4730923834537,POINT(33.5022 -86.8092)
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,tcc.noellevitz.com/(S(miwoihs5stz5cpyifh4nczu0...,...,0.775,0.610465116279,0.686567164179,0.632653061224,0.65,NaN,PrivacySuppressed,NaN,PrivacySuppressed,POINT(32.3626 -86.174)
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,finaid.uah.edu/,...,0.85140562249,0.768378650554,0.798440979955,0.778156996587,0.791666666667,NaN,0.47821134615384,NaN,0.49412939796716,POINT(34.7228 -86.6384)
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/email/index.aspx,www.alasu.edu/cost-aid/forms/calculator/index....,...,0.380829015544,0.364423717522,0.286530612245,0.315737051793,0.334633385335,NaN,0.25748195545488,NaN,0.29365162671232,POINT(32.3643 -86.2957)


### Selecting Colleges that provide relevant degrees and Data Cleaning

In [260]:
## Filtering df for college name, city name, and associate or graduate degrees
## From the data dictionary, we can see which columns tell us whether the degrees are offered or not
## These columns tell us in 0 or 1 terms if the degrees are offered or not

college = college[["instnm", "city", "cip11assoc", "cip11cert4"]]

# Performing a union of the two columns
college = college[(college["cip11assoc"] == 1) | (college["cip11cert4"] == 1)]

# Filtering for Metro cities
Metro_cities = ["Austin", "Miami", "San Jose", "Los Angeles", "Denver", "Las Vegas", "Houston", "San Francisco", "San Antonio", "Houston"]

college = college.loc[college["city"].isin(Metro_cities)]

college.shape

(44, 4)

# D. Merging Startup, Crime and College Dataframes

In [261]:
## Introducing crime ranks by merging the two dfs - college and crime.
## Introducing Startup activity ranks by merging the two dfs - college and crime.

college = college[college.columns].merge(crime, 'left')
college = college[college.columns].merge(startup, 'left')

college.head()

,instnm,city,cip11assoc,cip11cert4,Safety_Crime_Rank,Startup_Rank
0,Heald College-San Francisco,San Francisco,1.0,0.0,9,8.0
1,Los Angeles Southwest College,Los Angeles,1.0,0.0,3,4.0
2,Los Angeles Trade Technical College,Los Angeles,1.0,0.0,3,4.0
3,Los Angeles City College,Los Angeles,1.0,0.0,3,4.0
4,The Art Institute of California-Argosy Univers...,San Francisco,1.0,0.0,9,8.0


### Combining Crime Rank and Startup Rank in a single rank metric for final rank calculation

In [262]:
## Now, we have to determine the rank of these colleges, for which we need an metric
# Since low crime and high startup activity are equally important, we add up the two
# And then sort the df by the new metric

college["Rank_Score"] = (college["Safety_Crime_Rank"] + college["Startup_Rank"])
college = college.sort_values(by="Rank_Score", ascending=True)

college = college.reset_index()
college["Rank"] = college["Rank_Score"].index + 1

college.head()

,index,instnm,city,cip11assoc,cip11cert4,Safety_Crime_Rank,Startup_Rank,Rank_Score,Rank
0,21,ITT Technical Institute-Austin,Austin,1.0,0.0,1,1.0,2.0,1
1,38,The Art Institute of Austin,Austin,1.0,0.0,1,1.0,2.0,2
2,15,Concordia University-Texas,Austin,1.0,0.0,1,1.0,2.0,3
3,41,South University-Austin,Austin,1.0,0.0,1,1.0,2.0,4
4,12,Riverland Community College,Austin,2.0,1.0,1,1.0,2.0,5


In [263]:
college = college[["instnm", "city", "Rank_Score", "Rank"]]

college

,instnm,city,Rank_Score,Rank
0,ITT Technical Institute-Austin,Austin,2.0,1
1,The Art Institute of Austin,Austin,2.0,2
2,Concordia University-Texas,Austin,2.0,3
3,South University-Austin,Austin,2.0,4
4,Riverland Community College,Austin,2.0,5
5,Carrington College California-San Jose,San Jose,5.0,6
6,Argosy University-Los Angeles,Los Angeles,7.0,7
7,Advanced Computing Institute,Los Angeles,7.0,8
8,Los Angeles ORT College-Los Angeles Campus,Los Angeles,7.0,9
9,Los Angeles City College,Los Angeles,7.0,10


### Creating a temporary dataframe for city rank determination by duplicate removal

In [264]:
## Assigning a rank by first removing duplicates
# And then using df.index

Temp = college[["Rank_Score", "Rank"]]
Temp = Temp.drop_duplicates("Rank_Score")

Temp = Temp.reset_index()

Temp["Final_Rank"] = Temp["Rank_Score"].index + 1

Temp = Temp.drop(["index", "Rank"], axis=1)
Temp.columns = ["Rank_Score", "Final_Rank"]

Temp

,Rank_Score,Final_Rank
0,2.0,1
1,5.0,2
2,7.0,3
3,8.0,4
4,9.0,5
5,12.0,6
6,14.0,7
7,16.0,8
8,17.0,9


# E. Final Rank Calculation by merging Temp and college dataframes

In [265]:
## Introducing final ranks to the college df by merging Temp df and college df

college = college[college.columns].merge(Temp, 'left')

college = college.drop(["Rank", "Rank_Score"], axis=1)

college

,instnm,city,Final_Rank
0,ITT Technical Institute-Austin,Austin,1
1,The Art Institute of Austin,Austin,1
2,Concordia University-Texas,Austin,1
3,South University-Austin,Austin,1
4,Riverland Community College,Austin,1
5,Carrington College California-San Jose,San Jose,2
6,Argosy University-Los Angeles,Los Angeles,3
7,Advanced Computing Institute,Los Angeles,3
8,Los Angeles ORT College-Los Angeles Campus,Los Angeles,3
9,Los Angeles City College,Los Angeles,3


## The above are the colleges from which Maria should choose!!!